In [34]:
import pyodbc
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import shapiro

In [23]:
# Synapse GSK Prod

server   = "lana-sqlserver-prd-01.database.windows.net"
database = 'lana-synapse-prd-01'
driver    = '{ODBC Driver 17 for SQL Server}'
user = 'lct_9d24f4ccd9e64b7795e67089afbbc70c_prd_01_ro'
password = 'L0nx@0621gskro'
schema_name   = "lct_9d24f4ccd9e64b7795e67089afbbc70c_prd_01"
port = '1433'

#tenant_url = "https://ssplctcgauat01.jdadelivers.com"

def call_query(sql_query):
    conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+user+';PWD='+ password)
    cur = conn.cursor()
    data = pd.read_sql_query(sql_query,conn)
    return data  

In [29]:
%%time

### Testing with top 100 nodes based on revenue (sales orders)
### Latest version of Projected Inventoryis considered


SQL_1= """WITH filter_cte (
	derived_id,Revenue
	)
AS (
	SELECT TOP (3000) 
       b.derived_id 
      ,a.order_quantity * a.unit_price as 'Revenue'
  FROM [sales_order_line] a
  LEFT JOIN node_inventory b 
        ON a.supplier_item_name = b.customer_item_name
        AND a.ship_from_site_name = b. ship_to_site_name
  WHERE a. need_by_date > Dateadd(MONTH, -6, Getdate()) 
        AND a. need_by_date <= Dateadd(MONTH, 0, Getdate())
        AND a. line_status <> 'cancelled'
  ORDER BY   (a.order_quantity * a.unit_price) DESC

 )
	
	,pi_cte (
	node_inventory_derived_id
	,measure_date
	,pi
	)
AS (
	SELECT node_inventory_derived_id
		,measure_date
		,pi
	FROM (
		SELECT cast(node_inventory_derived_id AS CHAR) AS 'node_inventory_derived_id'
			,quantity AS 'pi'
			,cast(measure_date AS DATE) AS 'measure_date'
			,rank() OVER (
				PARTITION BY node_inventory_derived_id
				,measure_date ORDER BY version_date DESC
				) AS rank_
		FROM node_inventory_projected_inventory
		WHERE node_inventory_derived_id IN (
				SELECT [derived_id]
				FROM filter_cte
				)
			AND measure_date > Dateadd(MONTH, - 6, Getdate())
			AND measure_date <= Dateadd(day, 0, Getdate())
		) c
	WHERE rank_ = 1
	)
	,oh_cte (
	node_inventory_derived_id
	,measure_date
	,on_hand
	)
AS (
	SELECT node_inventory_derived_id
		,measure_date
		,on_hand
	FROM (
		SELECT cast(node_inventory_derived_id AS CHAR) AS 'node_inventory_derived_id'
			,sum(quantity) AS 'on_hand'
			,cast(measure_date AS DATE) AS 'measure_date'
		FROM node_inventory_on_hand
		WHERE node_inventory_derived_id IN (
				SELECT [derived_id]
				FROM filter_cte
				)
			AND measure_date > Dateadd(Month, - 6, Getdate())
			AND measure_date <= Dateadd(day, 0, Getdate())
		GROUP BY node_inventory_derived_id
			,measure_date
		) d
	)
	,join1_cte (
	node_inventory_derived_id
	,measure_date
	,on_hand
	,pi
	)
AS (
	SELECT COALESCE(a.node_inventory_derived_id, b.node_inventory_derived_id) AS 'node_inventory_derived_id'
		,COALESCE(a.measure_date, b.measure_date) AS measure_date
		,a.on_hand
		,b.pi
	FROM oh_cte a
	FULL OUTER JOIN pi_cte b ON a.node_inventory_derived_id = b.node_inventory_derived_id
		AND a.measure_date = b.measure_date
	)
	,join2_cte (
	node_inventory_derived_id
	,measure_date
	,pi
	,on_hand
	,variance
	,var_pc
	)
AS (
	SELECT a.node_inventory_derived_id
		,a.measure_date
		,a.[pi]
		,a.on_hand
		,(a.on_hand - a.pi) AS 'variance'
		,abs(a.on_hand - a.pi) / nullif(a.pi, 0) AS 'var_pc'
	FROM join1_cte a
	LEFT JOIN filter_cte b ON a.node_inventory_derived_id = b.derived_id
	)

SELECT *
FROM join2_cte
"""

RawData = call_query(SQL_1)
#RawData['url'] = tenant_url + RawData['url'].astype(str)
RawData.head()

Wall time: 58.3 s


,node_inventory_derived_id,measure_date,pi,on_hand,variance,var_pc
0,8142297137661886803,2021-08-09,147180.0,NaN,NaN,NaN
1,-6444109849161613975,2021-08-09,-121.0,NaN,NaN,NaN
2,-4809007790516659147,2021-08-09,0.0,NaN,NaN,NaN
3,-4208391052639564066,2021-08-13,33300.0,33300.0,0.0,0.0
4,-4493883922221608374,2021-08-09,40944.0,NaN,NaN,NaN


In [30]:
RawData.describe()

,pi,on_hand,variance,var_pc
count,5.304800e+05,3.344240e+05,3.341480e+05,331689.000000
mean,1.991722e+05,2.110421e+05,-2.852503e+04,0.213767
std,6.375360e+05,5.386656e+05,7.764084e+05,45.089821
min,-2.152308e+06,0.000000e+00,-1.300305e+07,-3962.821429
25%,0.000000e+00,1.192600e+04,-5.943000e+03,0.000000
50%,4.013700e+04,7.702000e+04,3.121000e+03,0.150793
75%,2.228150e+05,2.644200e+05,3.175200e+04,0.486903
max,1.370120e+07,3.998606e+07,4.065580e+07,5547.950000


#### Normality test

In [37]:
# The Shapiro-Wilk test is a statistical test used to check if a continuous variable follows a normal distribution. 
# The null hypothesis (H0) states that the variable is normally distributed, 
# and the alternative hypothesis (H1) states that the variable is NOT normally distributed.

#stats.shapiro(RawData['pi'])
shapiro(RawData['pi'])

ShapiroResult(statistic=nan, pvalue=1.0)

In [38]:
# test normality via shapiro
#stats.shapiro(RawData['on_hand'])
shapiro(RawData['on_hand'])

ShapiroResult(statistic=nan, pvalue=1.0)

#### Since we had a small sample size, determining the distribution of PI and On_hand was important for choosing appropriate statistical method. So, a Shapiro-Wilk test was performed and did not show evidence of non-normality (p-value>0.05). Based on this outcome, we decided to perform a Paired T-Test for hypothesis testing. This test allows us to evaluate differences between samples of the same group at different points in time, in this case, projected inventory (pi) and on_hand.

### Hypothesis Testing - Paired Sample T-Test
##### Null hypothesis, H0 : there is no difference between PI and On_hand (mean difference between the two is 0)
#####  Alternate hypothesis, H1 : there is difference between PI and On_hand (mean difference between the two is not 0)

In [39]:
stats.ttest_rel(RawData['pi'],RawData['on_hand'],nan_policy = 'omit')

Ttest_relResult(statistic=21.237594500946948, pvalue=5.001660851720675e-100)

#### A paired sample t-test was used to analyze projected inventory computation & on hand inventory to test if they are significantly different. The projected inventory computation was higher (199,172 +-637,537 units) compared to actual on_hand inventory (211,042 +-538,666); the findings are not statistical significant, rejecting null hypothesis.

#### There is difference between PI and On_hand in GSK Production